In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents.base import Document
from langgraph.graph import StateGraph, START, END
from typing import Annotated, Literal
from typing_extensions import TypedDict
from pydantic import BaseModel, Field
import dotenv
import os

dotenv.load_dotenv()

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini",
                 temperature=0.,)

embeddings = OpenAIEmbeddings()

In [ ]:
path = "./rules/"
filename = os.listdir(path)

In [ ]:
# 텍스트 스플리터 생성

splitter = RecursiveCharacterTextSplitter(chunk_size=100, 
                                          chunk_overlap=0,
                                          separators=["\n\n"])

In [ ]:
# PDF 로더 생성

hr_loader = PyPDFLoader(path+filename[0])
security_loader = PyPDFLoader(path+filename[1])
onboard_loader = PyPDFLoader(path+filename[2])
tools_loader = PyPDFLoader(path+filename[3])
culture_loader = PyPDFLoader(path+filename[4])

In [ ]:
# 문서 전처리 함수 생성

def cleaning_docs(docs):
    docs = docs.load()
    lens = None
    for idx, doc in enumerate(docs):
        corpus = doc.page_content.replace("\xa0", "").replace("  ", " ").split("\n")
        if lens is None:
            lens = []
            for sentence in corpus:
                lens.append(len(sentence))
            length = sorted(lens)[len(lens)//2]
        else:
            pass

        cleaning_corpus = []
        for sentence in corpus[:-2]:
            if len(sentence) >= length:
                cleaning_corpus.append(sentence)
            else:
                cleaning_corpus.append(sentence+"\n\n")   
        docs[idx].page_content = "".join(cleaning_corpus)

    return docs

In [ ]:
# 전처리 함수를 통해 문서를 전처리하세요.

hr_docs = # Your Code
security_docs = # Your Code
onboard_docs = # Your Code
tools_docs = # Your Code
culture_docs = # Your Code

In [ ]:
# 텍스트 스플리터를 이용한 문서 분할

hr_docs = splitter.split_documents(hr_docs)
security_docs = splitter.split_documents(security_docs)
onboard_docs = splitter.split_documents(onboard_docs)
tools_docs = splitter.split_documents(tools_docs)
culture_docs = splitter.split_documents(culture_docs)

In [ ]:
# 벡터스토어 생성

hr_vector_store = FAISS.from_documents(embedding=embeddings, documents=hr_docs)
security_vector_store = FAISS.from_documents(embedding=embeddings, documents=security_docs)
onboard_vector_store = FAISS.from_documents(embedding=embeddings, documents=onboard_docs)
tools_vector_store = FAISS.from_documents(embedding=embeddings, documents=tools_docs)
culture_vector_store = FAISS.from_documents(embedding=embeddings, documents=culture_docs)

In [ ]:
# 리트리버 생성

hr_retriever = hr_vector_store.as_retriever()
security_retriever = security_vector_store.as_retriever()
onboard_retriever = onboard_vector_store.as_retriever()
tools_retriever = tools_vector_store.as_retriever()
culture_retriever = culture_vector_store.as_retriever()

compressor = LLMChainExtractor.from_llm(llm)

In [ ]:
# LLM 기반 Reranker

hr_reranked = ContextualCompressionRetriever(
    base_retriever=hr_retriever,
    base_compressor=compressor
)

security_reranked = ContextualCompressionRetriever(
    base_retriever=security_retriever,
    base_compressor=compressor
)

onboard_reranked = ContextualCompressionRetriever(
    base_retriever=onboard_retriever,
    base_compressor=compressor
)

tools_reranked = ContextualCompressionRetriever(
    base_retriever=tools_retriever,
    base_compressor=compressor
)

culture_reranked = ContextualCompressionRetriever(
    base_retriever=culture_retriever,
    base_compressor=compressor
)

In [ ]:
class State(TypedDict):
    query : Annotated[str, "User Question"]
    answer : Annotated[str, "LLM response"]
    document : Annotated[Document, "Retrieve Response"]
    retrieval_type : Annotated[str, "Document Category"]

In [ ]:
class RetriverChecker(BaseModel):
    """
    질문의 의도를 파악하고 5가지 주제 중 어디에 속하는지 답변합니다.
    당신이 가진 주제는 아래와 같습니다.

    1. 인사 운영 메뉴얼 : 회사의 인사 운영 원칙과 절차를 담은 매뉴얼입니다. 채용, 원격근무, 근로시간, 휴가, 평가, 복지, 퇴직 등 직원 전반의 라이프사이클을 공정하고 효율적으로 관리하기 위한 기준을 안내합니다.
    2. 보안 정책 : 회사의 정보 자산과 고객 데이터를 안전하게 보호하기 위한 보안 정책입니다. 계정 관리, 데이터 보안, 물리적 보안, 사고 대응, 보안 교육 등 전사적 보안 수칙을 담아 모든 임직원과 협력사가 따라야 할 기준을 안내합니다.
    3. 온보딩 메뉴얼 : 신규 입사자가 조직에 빠르게 적응하고 성과를 낼 수 있도록 돕는 온보딩 가이드입니다. 입사 전 준비부터 첫 3개월간의 일정, 교육, 피드백, 문화 적응까지 체계적인 지원 절차를 제공합니다.
    4. 업무 도구 가이드 : 회사에서 사용하는 주요 협업 도구의 사용 원칙과 규칙을 정리한 가이드입니다. 일관된 커뮤니케이션과 효율적인 협업을 위한 도구별 활용법과 팀 내 운영 기준을 안내합니다.
    5. 문화 규칙 : 우리 팀이 자연스럽게 지키는 협업과 소통의 문화 원칙입니다. 시간 약속, 수평적 호칭, 명확한 소통, 유대감 있는 잡담, 자율적 휴식, 책임 있는 결정 등 모두가 함께 일하기 좋은 팀 문화를 위한 10가지 약속을 담고 있습니다.

    질문이 1에 해당한다면 "HR", 2에 해당한다면 "Security", 3에 해당한다면 "Onboard", 4에 해당한다면 "Tools", 5에 해당한다면 "Culture"라는 답변을 반환합니다.

    """

    retrieval_type : Literal["HR", "Security", "Onboard", "Tools", "Culture"] = Field(..., description="""Identify the intent of the question and answer which of the five topics it belongs to.
The topics you have are as follows.

1. Personnel Management Manual: This is a manual that contains the company's personnel management principles and procedures. It guides the standards for fair and efficient management of the overall lifecycle of employees such as hiring, remote work, working hours, vacation, evaluation, welfare, and retirement.
2. Security Policy: A security policy to secure the company's information assets and customer data. It guides all executives and partners to follow with company-wide security rules such as account management, data security, physical security, incident response, and security training.
3. Onboarding Manual: This is an onboarding guide that helps new employees quickly adapt to the organization and achieve results. It provides a systematic support process from pre-employment preparation to the first three months of scheduling, training, feedback, and cultural adaptation.
4. Work Tool Guide: A guide that outlines the principles and rules of use of key collaborative tools used by the company. It guides you on how to use each tool for consistent communication and efficient collaboration and how to operate within the team.
5. Cultural Rules: The cultural principles of collaboration and communication that our team naturally follows. They contain 10 commitments for a team culture that is good for everyone to work with: time commitments, horizontal calling, clear communication, bonding small talk, autonomous rest, responsible decisions, etc.

Return "HR" if the question corresponds to 1, "Security" if it corresponds to 2, "Onboard" if it corresponds to 3, "Tools" if it corresponds to 4, and "Culture" if it corresponds to 5.
    Return "yes" if you can answer, "no" if you can't answer.""")

In [ ]:
retriever_checker = llm.with_structured_output(RetriverChecker)

In [ ]:
result = retriever_checker.invoke("복지에는 어떤 것들이 있나요?")

In [ ]:
result

In [ ]:
def retriever_check(state: State):
    prompt = PromptTemplate.from_template(
    """
    질문의 의도를 파악하고 5가지 주제 중 어디에 속하는지 답변합니다.
    당신이 가진 주제는 아래와 같습니다.

    1. 인사 운영 메뉴얼 : 회사의 인사 운영 원칙과 절차를 담은 매뉴얼입니다. 채용, 원격근무, 근로시간, 휴가, 평가, 복지, 퇴직 등 직원 전반의 라이프사이클을 공정하고 효율적으로 관리하기 위한 기준을 안내합니다.
    2. 보안 정책 : 회사의 정보 자산과 고객 데이터를 안전하게 보호하기 위한 보안 정책입니다. 계정 관리, 데이터 보안, 물리적 보안, 사고 대응, 보안 교육 등 전사적 보안 수칙을 담아 모든 임직원과 협력사가 따라야 할 기준을 안내합니다.
    3. 온보딩 메뉴얼 : 신규 입사자가 조직에 빠르게 적응하고 성과를 낼 수 있도록 돕는 온보딩 가이드입니다. 입사 전 준비부터 첫 3개월간의 일정, 교육, 피드백, 문화 적응까지 체계적인 지원 절차를 제공합니다.
    4. 업무 도구 가이드 : 회사에서 사용하는 주요 협업 도구의 사용 원칙과 규칙을 정리한 가이드입니다. 일관된 커뮤니케이션과 효율적인 협업을 위한 도구별 활용법과 팀 내 운영 기준을 안내합니다.
    5. 문화 규칙 : 우리 팀이 자연스럽게 지키는 협업과 소통의 문화 원칙입니다. 시간 약속, 수평적 호칭, 명확한 소통, 유대감 있는 잡담, 자율적 휴식, 책임 있는 결정 등 모두가 함께 일하기 좋은 팀 문화를 위한 10가지 약속을 담고 있습니다.

    질문이 1에 해당한다면 "HR", 2에 해당한다면 "Security", 3에 해당한다면 "Onboard", 4에 해당한다면 "Tools", 5에 해당한다면 "Culture"라는 답변을 반환합니다.

    질문 : {query}

    """
    )

    chain = # 체인을 구성해주세요.

    result = chain.invoke({"query":state["query"]})

    return {"retrieval_type" : result.retrieval_type}

In [ ]:
def reranker(state: State):

    if state["retrieval_type"] == "HR":
        docs = hr_reranked.invoke(state["query"])
        return # state의 document에 docs를 반환해주세요.
    elif state["retrieval_type"] == "Security":
        docs = security_reranked.invoke(state["query"])
        return # state의 document에 docs를 반환해주세요.
    elif state["retrieval_type"] == "Onboard":
        docs = onboard_reranked.invoke(state["query"])
        return # state의 document에 docs를 반환해주세요.
    elif state["retrieval_type"] == "Tools":
        docs = tools_reranked.invoke(state["query"])
        return # state의 document에 docs를 반환해주세요.
    else:
        docs = culture_reranked.invoke(state["query"])
        return # state의 document에 docs를 반환해주세요.

In [ ]:
def response(state: State):
    prompt = ChatPromptTemplate([
        ("system", "당신은 회사 내규 챗봇입니다. 사용자 정보와 회사 내규 문서가 주어집니다. 그것을 통해 사용자의 행동을 제시하세요.\n"
                "---"
                "문서 : {context}\n\n"
                "문서에서 응답을 찾을 수 없는 경우 '문서에서 응답을 찾을 수 없습니다.' 라고 답변하세요."), 
        ("user", "{query}")
        ])
    
    docs = "\n\n".join(doc.page_content for doc in state["document"])
    
    chain = prompt | llm

    result = chain.invoke({# context는 docs를 query는 state의 query를 입력으로 받습니다.})
    
    return {"answer":result}

In [ ]:
graph_builder = StateGraph(State)

In [ ]:
# 노드와 엣지를 구성하세요.

In [ ]:
graph = graph_builder.compile()

In [ ]:
graph

In [ ]:
result = graph.stream({"query": "우리 회사에는 어떤 복지제도가 있나요?"})

In [ ]:
for step in result:
    for k,v in step.items():
        print(f"\n\n=== {k} ===\n\n")
        if k != "response":
            print(v)
        else:
            print(v["answer"].content)

In [ ]:
result = graph.invoke({"query": "원격근무는 언제 할 수 있나요?"})

In [ ]:
result

In [ ]:
result = graph.stream({"query": "퇴사를 계획중인데 어떻게 하면 되나요?"})

In [ ]:
for step in result:
    for k,v in step.items():
        print(f"\n\n=== {k} ===\n\n")
        if k != "response":
            print(v)
        else:
            print(v["answer"].content)

In [ ]:
result = graph.stream({"query": "회사 계정의 비밀번호는 어떤 규칙대로 만들어야하나요?"})

In [ ]:
for step in result:
    for k,v in step.items():
        print(f"\n\n=== {k} ===\n\n")
        if k != "response":
            print(v)
        else:
            print(v["answer"].content)